### import

In [1]:
import numpy as np 
import pandas as pd 
pd.set_option('display.max_columns', None)
from sklearn.preprocessing import StandardScaler, OneHotEncoder 
import pickle 

### 데이터 전처리 클래스 및 함수 정의

In [2]:
# 추후 날씨 데이터의 위도 경도를 county로 변환하기 위한 데이터
location = [
    [0.0, 24.2, 59.1],
    [0.0, 25.2, 59.1],
    [0.0, 23.7, 59.4],
    [0.0, 24.2, 59.4],
    [0.0, 24.7, 59.4],
    [0.0, 25.2, 59.4],
    [0.0, 25.7, 59.4],
    [0.0, 24.7, 59.7],
    [0.0, 25.2, 59.7],
    [0.0, 25.7, 59.7],
    [1.0, 21.7, 58.8],
    [1.0, 22.2, 58.8],
    [1.0, 22.7, 58.8],
    [1.0, 23.2, 58.8],
    [1.0, 22.2, 59.1],
    [1.0, 22.7, 59.1],
    [2.0, 27.2, 59.1],
    [2.0, 27.7, 59.1],
    [2.0, 27.2, 59.4],
    [2.0, 27.7, 59.4],
    [3.0, 25.2, 58.8],
    [3.0, 25.7, 58.8],
    [3.0, 25.7, 59.1],
    [4.0, 26.2, 58.5],
    [4.0, 26.2, 58.8],
    [4.0, 26.7, 58.8],
    [4.0, 27.2, 58.8],
    [5.0, 26.2, 59.1],
    [5.0, 26.7, 59.1],
    [5.0, 26.2, 59.4],
    [5.0, 26.7, 59.4],
    [5.0, 26.2, 59.7],
    [5.0, 26.7, 59.7],
    [6.0, 23.7, 58.8],
    [6.0, 23.2, 59.1],
    [6.0, 23.7, 59.1],
    [6.0, 23.2, 59.4],
    [7.0, 23.7, 57.9],
    [7.0, 24.2, 57.9],
    [7.0, 23.7, 58.2],
    [7.0, 24.2, 58.2],
    [7.0, 24.7, 58.2],
    [7.0, 25.2, 58.2],
    [7.0, 23.7, 58.5],
    [7.0, 24.2, 58.5],
    [7.0, 24.7, 58.5],
    [8.0, 27.2, 58.2],
    [9.0, 24.2, 58.8],
    [9.0, 24.7, 58.8],
    [9.0, 24.7, 59.1],
    [10.0, 23.2, 57.6],
    [10.0, 21.7, 57.9],
    [10.0, 22.2, 57.9],
    [10.0, 23.2, 57.9],
    [10.0, 21.7, 58.2],
    [10.0, 22.2, 58.2],
    [10.0, 22.7, 58.2],
    [10.0, 23.2, 58.2],
    [10.0, 21.7, 58.5],
    [10.0, 22.2, 58.5],
    [10.0, 22.7, 58.5],
    [10.0, 23.2, 58.5],
    [11.0, 26.2, 58.2],
    [11.0, 26.7, 58.2],
    [11.0, 26.7, 58.5],
    [11.0, 27.2, 58.5],
    [13.0, 26.2, 57.9],
    [14.0, 25.7, 58.2],
    [14.0, 25.2, 58.5],
    [14.0, 25.7, 58.5],
    [15.0, 26.7, 57.6],
    [15.0, 27.2, 57.6],
    [15.0, 26.7, 57.9],
    [15.0, 27.2, 57.9],
    [15.0, 27.7, 57.9]
]

location = pd.DataFrame(location, columns=['county', 'longitude', 'latitude'])

In [3]:
class FeatureProcessorClass():
    def __init__(self):
        # join 기준
        self.weather_join = ['datetime', 'county', 'data_block_id']
        self.gas_join = ['data_block_id']
        self.electricity_join = ['datetime', 'data_block_id']
        self.client_join = ['county', 'is_business', 'product_type', 'data_block_id']

        # 위도 경도 칼럼
        self.lat_lon_columns = ['latitude', 'longitude']

        # 추가할 Aggregate stats
        self.agg_stats = ['mean'] #, 'min', 'max', 'std', 'median']
    
    def create_new_column_names(self, df, suffix, columns_no_change):
        # 칼럼명 변경
        df.columns = [col + suffix
                      if col not in columns_no_change
                      else col
                      for col in df.columns
                      ]
        return df

    def flatten_multi_index_columns(self, df):
        df.columns = ['_'.join([col for col in multi_col if len(col)>0])
                      for multi_col in df.columns]
        return df

    def create_data_features(self, data):
        # 날짜 관련 feature
        data['datetime'] = pd.to_datetime(data['datetime'])
        data['date'] = data['datetime'].dt.normalize()
        data['year'] = data['datetime'].dt.year
        data['quarter'] = data['datetime'].dt.quarter
        data['month'] = data['datetime'].dt.month
        data['week'] = data['datetime'].dt.isocalendar().week
        data['hour'] = data['datetime'].dt.hour
        data['day_of_year'] = data['datetime'].dt.day_of_year
        data['day_of_month']  = data['datetime'].dt.day
        data['day_of_week'] = data['datetime'].dt.day_of_week
        return data

    def create_client_features(self, client):
        # client 칼럼명 변경
        client = self.create_new_column_names(client,
                                           suffix='_client',
                                           columns_no_change = self.client_join
                                          )
        return client

    def create_historical_weather_features(self, historical_weather):
        # To datetime
        historical_weather['datetime'] = pd.to_datetime(historical_weather['datetime'])

        # 위도 경도를 county로 변환
        historical_weather[self.lat_lon_columns] = historical_weather[self.lat_lon_columns].astype(float).round(1)
        historical_weather = historical_weather.merge(location, how = 'left', on = self.lat_lon_columns)
        # historical_weather 칼럼명 변경
        historical_weather = self.create_new_column_names(historical_weather,
                                                          suffix='_h',
                                                          columns_no_change = self.lat_lon_columns + self.weather_join
                                                          )

        # Aggregate stats 계산
        agg_columns = [col for col in historical_weather.columns if col not in self.lat_lon_columns + self.weather_join]
        agg_dict = {agg_col: self.agg_stats for agg_col in agg_columns}
        historical_weather = historical_weather.groupby(self.weather_join).agg(agg_dict).reset_index()

        historical_weather = self.flatten_multi_index_columns(historical_weather)

        # 하루씩 밀기
        historical_weather['hour_h'] = historical_weather['datetime'].dt.hour
        historical_weather['datetime'] = (historical_weather
                                               .apply(lambda x:
                                                      x['datetime'] + pd.DateOffset(1)
                                                      if x['hour_h']< 11
                                                      else x['datetime'] + pd.DateOffset(2),
                                                      axis=1)
                                              )

        return historical_weather

    def create_forecast_weather_features(self, forecast_weather):
        # forecast_weather 칼럼명 변경
        forecast_weather = (forecast_weather
                            .rename(columns = {'forecast_datetime': 'datetime'})
                            .drop(columns = 'origin_datetime')
                           )

        # To datetime
        forecast_weather['datetime'] = (pd.to_datetime(forecast_weather['datetime'])
                                        .dt
                                        .tz_convert('Europe/Brussels')
                                        .dt
                                        .tz_localize(None)
                                       )

        # 위도 경도를 county로 변환
        forecast_weather[self.lat_lon_columns] = forecast_weather[self.lat_lon_columns].astype(float).round(1)
        forecast_weather = forecast_weather.merge(location, how = 'left', on = self.lat_lon_columns)

        # forecast_weather 칼럼명 변경
        forecast_weather = self.create_new_column_names(forecast_weather,
                                                        suffix='_f',
                                                        columns_no_change = self.lat_lon_columns + self.weather_join
                                                        )

        # Aggregate stats 계산
        agg_columns = [col for col in forecast_weather.columns if col not in self.lat_lon_columns + self.weather_join]
        agg_dict = {agg_col: self.agg_stats for agg_col in agg_columns}
        forecast_weather = forecast_weather.groupby(self.weather_join).agg(agg_dict).reset_index()

        forecast_weather = self.flatten_multi_index_columns(forecast_weather)
        return forecast_weather

    def create_electricity_features(self, electricity):
        # To datetime
        electricity['forecast_date'] = pd.to_datetime(electricity['forecast_date'])

        # 하루씩 밀기
        electricity['datetime'] = electricity['forecast_date'] + pd.DateOffset(1)

        # electricity 칼럼명 변경
        electricity = self.create_new_column_names(electricity,
                                                   suffix='_electricity',
                                                   columns_no_change = self.electricity_join
                                                  )
        return electricity

    def create_gas_features(self, gas):
        # Mean 계산
        gas['mean_price_per_mwh'] = (gas['lowest_price_per_mwh'] + gas['highest_price_per_mwh'])/2

        # gas 칼럼명 변경
        gas = self.create_new_column_names(gas,
                                           suffix='_gas',
                                           columns_no_change = self.gas_join
                                          )
        return gas

    def __call__(self, data, client, historical_weather, forecast_weather, electricity, gas):
        # 정의한 메소드들로 feature 생성
        data = self.create_data_features(data)
        client = self.create_client_features(client)
        historical_weather = self.create_historical_weather_features(historical_weather)
        forecast_weather = self.create_forecast_weather_features(forecast_weather)
        electricity = self.create_electricity_features(electricity)
        gas = self.create_gas_features(gas)

        # 하나의 df에 모두 통합
        df = data.merge(client, how='left', on = self.client_join)
        df = df.merge(historical_weather, how='left', on = self.weather_join)
        df = df.merge(forecast_weather, how='left', on = self.weather_join)
        df = df.merge(electricity, how='left', on = self.electricity_join)
        df = df.merge(gas, how='left', on = self.gas_join)

        return df

In [4]:
def create_revealed_targets_train(data):
    # day_lag 만큼의 과거 target 값을 feature로 생성
    original_datetime = data['datetime']
    revealed_targets = data[['datetime', 'prediction_unit_id', 'is_consumption', 'target']].copy()

    day_lag = 2 # kaggle competition에서는 2일 전의 target 값을 feature로 사용
    revealed_targets['datetime'] = original_datetime + pd.DateOffset(day_lag)
    data = data.merge(revealed_targets,
                        how='left',
                        on = ['datetime', 'prediction_unit_id', 'is_consumption'],
                        suffixes = ('', f'_{day_lag}_days_ago')
                        )
    return data

In [5]:
def data_preprocessing(train, client, historical_weather, forecast_weather, electricity_prices, gas_prices):
    data = train.copy()
    client = client.copy()
    historical_weather = historical_weather.copy()
    forecast_weather = forecast_weather.copy()
    electricity = electricity_prices.copy()
    gas = gas_prices.copy()
    
    FeatureProcessor = FeatureProcessorClass()
    
    data = FeatureProcessor(data,
                      client,
                      historical_weather,
                      forecast_weather,
                      electricity,
                      gas
                     )
    df = create_revealed_targets_train(data)

    # Remove columns for features\
    no_features = ['date',
                    'latitude',
                    'longitude',
                    'hours_ahead',
                    'hour_h',
                    'prediction_unit_id',
                    'data_block_id',
                    'currently_scored',
                    'row_id'
                   ]

    remove_columns = [col for col in df.columns for no_feature in no_features if no_feature in col]
    features = [col for col in df.columns if col not in remove_columns]
    
    df = df[features]

    # backfill을 통해 결측치 처리
    df = df.bfill()
    
    ## Numerical Feauture Standard Scaling
    features_not_to_scale = ['county',
                              'is_business',
                              'product_type',
                              'is_consumption',
                              'year',
                              'quarter',
                              'month',
                              'week',
                              'hour',
                              'day_of_year',
                              'day_of_month',
                              'day_of_week',
                              'target']
    features_to_scale = [col for col in df.columns if col not in features_not_to_scale]
    
    Scaler = StandardScaler().fit(df[features_to_scale])
    
    # 추후 모델 활용을 위해 Scaler 저장
    with open('Scaler_DNN.pkl', 'wb') as file:
        pickle.dump(Scaler, file)

    df[features_to_scale] = Scaler.transform(df[features_to_scale])
    
    
    ## Categoical Feature One-hot Encoding
    df = df.reset_index(drop=True)
    
    columns_to_onehot_encode = df[['county', 'product_type']]

    Encoder = OneHotEncoder(sparse_output=False)
    encoded = Encoder.fit_transform(columns_to_onehot_encode)
    encoded_df = pd.DataFrame(encoded, columns=Encoder.get_feature_names_out(['county', 'product_type']))

    df = pd.concat([df, encoded_df], axis=1)
    
    df.drop(['county', 'product_type'], axis=1, inplace=True)

    return df
    

###  실제 Train 데이터 불러와서 전처리

In [6]:
file_path = "data/"

train = pd.read_csv(file_path + 'train.csv', parse_dates=['datetime'])
client = pd.read_csv(file_path + 'client.csv')
electricity = pd.read_csv(file_path + 'electricity_prices.csv')
forecast_weather = pd.read_csv(file_path + 'forecast_weather.csv')
gas = pd.read_csv(file_path + 'gas_prices.csv')
historical_weather = pd.read_csv(file_path + 'historical_weather.csv')

In [7]:
print(train.isnull().sum())

#target 결측치 모두 특정 날짜의 새벽 3am 값 --> 전날 target값으로 대체
missing_data = train[train.isna().any(axis=1)]
dt = missing_data['datetime'].unique()
for i in dt:
   idx_to_process = train[train['datetime']== i].index
   ii = i-pd.Timedelta(days=1)
   train.loc[idx_to_process,'target'] = train[train['datetime']== ii]['target'].values

print(train.isnull().sum())

county                  0
is_business             0
product_type            0
target                528
is_consumption          0
datetime                0
data_block_id           0
row_id                  0
prediction_unit_id      0
dtype: int64
county                0
is_business           0
product_type          0
target                0
is_consumption        0
datetime              0
data_block_id         0
row_id                0
prediction_unit_id    0
dtype: int64


In [8]:
df = data_preprocessing(train, client, historical_weather, forecast_weather, electricity, gas)

In [9]:
# 전처리된 데이터 저장
df.to_csv('data_preprocessed.csv', index=False)

In [10]:
df = pd.read_csv('data_preprocessed.csv')

In [11]:
## 학습용, 검증용, 시험용으로 분리
from sklearn.model_selection import train_test_split

# 64:16:20으로 분리
X_train, X_test = train_test_split(df, test_size=0.2)
X_train, X_val = train_test_split(X_train, test_size=0.2)

y_train = X_train.pop('target')
y_val = X_val.pop('target')
y_test = X_test.pop('target')

### 모델 학습

In [12]:
from tensorflow.keras.layers import Dense, Input
# perceptron, flatten 해주는 라이브러리 불러오기
from tensorflow.keras.models import Sequential
# Sequential 하게 모델을 연결해주는 라이브러리 불러오기
from tensorflow.keras.optimizers.legacy import Adam

model = Sequential([
    Input(63),
    Dense(100, activation='relu'),
    Dense(300, activation='relu'),
    Dense(600, activation='relu'),
    Dense(300, activation='relu'),
    Dense(100, activation='relu'),
    Dense(60, activation='relu'),
    Dense(30, activation='relu'),
    Dense(10, activation='relu'),
    Dense(1, name='output')
])
model.compile(optimizer=Adam(0.001), loss='mae')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               6400      
                                                                 
 dense_1 (Dense)             (None, 300)               30300     
                                                                 
 dense_2 (Dense)             (None, 600)               180600    
                                                                 
 dense_3 (Dense)             (None, 300)               180300    
                                                                 
 dense_4 (Dense)             (None, 100)               30100     
                                                                 
 dense_5 (Dense)             (None, 60)                6060      
                                                                 
 dense_6 (Dense)             (None, 30)                1

In [13]:
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

mcp = ModelCheckpoint(filepath='weights/weight_{epoch:02d}_{val_loss:.2f}.hdf5', monitor='val_loss',
                        save_best_only=True, save_weights_only=True, mode='min', save_freq='epoch', verbose=0)
rlr = ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=10, mode='min', verbose=1)
ely = EarlyStopping(monitor='val_loss', patience=20, mode='min', verbose=1)

In [14]:
X_train = np.asarray(X_train).astype('float32')
y_train = np.asarray(y_train).astype('float32')
X_val = np.asarray(X_val).astype('float32')
y_val = np.asarray(y_val).astype('float32')

history = model.fit(x=X_train, y=y_train, batch_size=256, epochs=200, validation_data=(X_val, y_val),
                   callbacks=[mcp, rlr, ely])

Epoch 1/200
5046/5046 [==============================] - 31s 6ms/step - loss: 145.0991 - val_loss: 99.4330 - lr: 0.0010
Epoch 2/200
5046/5046 [==============================] - 33s 7ms/step - loss: 92.7078 - val_loss: 82.3034 - lr: 0.0010
Epoch 3/200
5046/5046 [==============================] - 34s 7ms/step - loss: 80.4886 - val_loss: 73.9223 - lr: 0.0010
Epoch 4/200
5046/5046 [==============================] - 36s 7ms/step - loss: 70.2152 - val_loss: 65.1399 - lr: 0.0010
Epoch 5/200
5046/5046 [==============================] - 38s 8ms/step - loss: 65.2686 - val_loss: 62.7076 - lr: 0.0010
Epoch 6/200
5046/5046 [==============================] - 41s 8ms/step - loss: 61.7953 - val_loss: 60.1300 - lr: 0.0010
Epoch 7/200
5046/5046 [==============================] - 41s 8ms/step - loss: 59.7763 - val_loss: 55.8724 - lr: 0.0010
Epoch 8/200
5046/5046 [==============================] - 40s 8ms/step - loss: 57.7377 - val_loss: 54.9474 - lr: 0.0010
Epoch 9/200
5046/5046 [========================

### 결과 및 모델 세이브

In [15]:
X_test = np.asarray(X_test).astype('float32')
y_test = np.asarray(y_test).astype('float32')
model.evaluate(X_test, y_test)

12615/12615 [==============================] - 11s 873us/step - loss: 32.5031


32.50313186645508

In [16]:
model.save_weights("DNN_model_weights.h5")